In [9]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [21]:
def url_parser(sDate, eDate, row=10):
    key="5oaL5XF7nxzVRPO3HpIwVCoC4CQBQx9tcB5WqEJnT%2FFA9KUyD7HAktzqQFC6xRqGSstXpp%2BSNVA6MjfX6%2BJlEQ%3D%3D"
    url="http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson?serviceKey="+key
    url=url+'&pageNo=1&numOfRows='+str(row)
    url=url+'&startCreateDt='+str(sDate)
    url=url+'&endCreateDt='+str(eDate)
    
    xml_soup=requests.get(url)
    if xml_soup.status_code != 200:
        print('API 연결 오류')
        exit()
    soup=bs(xml_soup.text, 'html.parser')
    
    return soup

In [25]:
def covid_info(soup):
    items=soup.find_all('item')
    covid_list=[]
    for item in items:
        stdDay=item.find('stdday').get_text()       #기준일
        gubun=item.find('gubun').get_text()         #지역명
        incDec=item.find('incdec').get_text()       #당일 확진자
        defCnt=item.find('defcnt').get_text()       # 누적 확진자
        localCnt=item.find('localocccnt').get_text() #지역 
        overCnt=item.find('overflowcnt').get_text()  #해외
        
        covid_list.append([stdDay,gubun,incDec,defCnt,localCnt,overCnt])
        
    df1=pd.DataFrame(covid_list, columns=['기준일', '지역명', '당일','누적', '지역','해외'])
    
    return df1

In [26]:
sDate='20210101'
eDate=datetime.datetime.today().strftime('%Y%m%d')



soup=url_parser(sDate, eDate)
row=soup.find('totalcount').text
soup=url_parser(sDate, eDate, row)
df1=covid_info(soup)

In [27]:
df1

,기준일,지역명,당일,누적,지역,해외
0,2021년 07월 14일 00시,검역,19,4593,0,19
1,2021년 07월 14일 00시,제주,21,1433,21,0
2,2021년 07월 14일 00시,경남,89,5661,87,2
3,2021년 07월 14일 00시,경북,22,5085,19,3
4,2021년 07월 14일 00시,전남,6,1744,6,0
...,...,...,...,...,...,...
3700,2021년 01월 01일 00시,인천,65,2904,63,2
3701,2021년 01월 01일 00시,대구,43,7844,43,0
3702,2021년 01월 01일 00시,부산,57,1925,55,2
3703,2021년 01월 01일 00시,서울,358,19362,357,1
